In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor#, GradientBoostingRegressor, ExtraTreesRegressor
# import xgboost as xgb
import pickle
import plotly.graph_objects as go

In [36]:
monthly_data=pd.read_csv("Monthly_Training_Data.csv",usecols=["Service Name","Month","Year","Count"])

In [37]:
with open('operations_names_en_to_ar.pickle', 'rb') as f:
    operations_names_en_to_ar = pickle.load(f)

In [38]:
def encode_monthly_names(df):
    Service_Names_to_index = {name : idx for idx, name in enumerate(monthly_data['Service Name'].unique())}
    df['Service Name']=df['Service Name'].map(Service_Names_to_index)
    return df

In [39]:
def decode_monthly_names(df):
    Service_Index_to_name= {idx : name for idx, name in enumerate(monthly_data['Service Name'].unique())}
    df['Service Name']=df['Service Name'].map(Service_Index_to_name)
    return df

In [40]:
# weekly_processed_data = weekly_data.copy()
# weekly_processed_data=encode_weekly_names(weekly_processed_data)
monthly_processed_data= monthly_data.copy()
monthly_processed_data=encode_monthly_names(monthly_processed_data)
monthly_processed_data.head()

,Service Name,Count,Month,Year
0,0,1,1,2016
1,1,16,1,2016
2,2,1,1,2016
3,3,40,1,2016
4,4,8,1,2016


In [41]:
# X=data.drop(["Count"],axis=1)
# y=data["Count"]

In [42]:
monthly_split_index = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] == 6)].iloc[-1].name
# weekly_split_index = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 11)].iloc[-1].name

print(monthly_split_index)

1158


In [43]:
# X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)
def get_X_y(df,split_index):
    X_train = df.iloc[:split_index].drop('Count', axis=1)
    X_test = df.iloc[split_index:].drop('Count', axis=1)

    y_train = df.iloc[:split_index]['Count']
    y_test = df.iloc[split_index:]['Count']

    return X_train, X_test, y_train, y_test


In [44]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [45]:
def monthly_prediction_cases(regressor):
     month_ahead = monthly_processed_data[:]
     
     # quarter_year = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] >= 9)]
     
     # half_year = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] >= 7)]

     month_ahead_pred = regressor.predict(month_ahead.drop('Count', axis=1))
     # quarter_year_pred = regressor.predict(quarter_year.drop('Count', axis=1))
     # half_year_pred = regressor.predict(half_year.drop('Count', axis=1))
     
     err = mean_absolute_error(month_ahead['Count'], month_ahead_pred)
     month_MAPE = mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)
     # quarter_year_MAPE = mean_absolute_percentage_error( quarter_year['Count'], quarter_year_pred)
     # half_year_MAPE = mean_absolute_percentage_error(half_year['Count'], half_year_pred) 
     
     month_ahead["predictions"]=month_ahead_pred
     # quarter_year["predictions"]=quarter_year_pred
     # half_year["predictions"]=half_year_pred

     month_ahead=decode_monthly_names(month_ahead)
     # quarter_year=decode_monthly_names(quarter_year)
     # half_year=decode_monthly_names(half_year)
     # month_ahead['Service Name']=month_ahead['Service Name'].map(Service_Index_to_name)
     # quarter_year['Service Name']=quarter_year['Service Name'].map(Service_Index_to_name)
     # half_year['Service Name']=half_year['Service Name'].map(Service_Index_to_name)
     

     month_ahead['Service Name AR']=month_ahead['Service Name'].map(operations_names_en_to_ar)
     # quarter_year['Service Name AR']=quarter_year['Service Name'].map(operations_names_en_to_ar)
     # half_year['Service Name AR']=half_year['Service Name'].map(operations_names_en_to_ar)     

     return err,month_MAPE, month_ahead

In [46]:
def perform_monthly_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    X_train, X_test, y_train, y_test=get_X_y(monthly_processed_data,monthly_split_index)
    regressor.fit(X_train, y_train)
    # y_pred = regressor.predict(X_test)
    # print(f'Total MAE {mean_absolute_percentage_error(y_pred, y_test)}')
    err,month_MAPE, month_ahead = monthly_prediction_cases(regressor)

    print(f'full data predection MAE {err}')
    print(f'full data predection MAPE {month_MAPE}')
    # print(f'Last quarter year MAPE {quarter_year_MAPE}')
    # print(f'half year MAPE {half_year_MAPE}')
    return month_ahead

In [47]:
def vis(df):
    fig = go.Figure(data=[
                        go.Bar(name = 'Real Data', x=df["Service Name"], y=df["Count"], marker_color='#800000'),
                        go.Bar(name = 'Predicted', x=df["Service Name"], y=df["predictions"], marker_color='#228B22'),

    ])

    fig.update_layout(barmode='group',width=1200, height=400)
    return fig.show()

In [48]:
with open('./ada_regressor.pickle', 'rb') as f:
  reg = pickle.load(f)
prediction=perform_monthly_regression(reg)

full data predection MAE 1.6012931034482758
full data predection MAPE 8.382439262314634


In [49]:
vis(prediction)

In [50]:
prediction.to_csv("Pred_Full_Data_ada.csv",encoding='utf-8-sig')
# month[0].to_csv("pred_one_month_ada.csv",encoding='utf-8-sig')
# quarter.to_csv("pred_last_quarter_ada.csv",encoding='utf-8-sig')
# half_year.to_csv("pred_half_year_ada.csv",encoding='utf-8-sig')